<a href="https://colab.research.google.com/github/havihaviplants/Artbox/blob/main/%EC%8B%A0%EA%B2%BD%EB%A7%9D_%ED%95%99%EC%8A%B5_%EB%B0%A9%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F

# 자신의 Google Drive 마운트하는 코드를 추가하자!
mnist = np.load('/content/drive/MyDrive/MY Drive/mnist.npz')
x_train = (mnist['x_train'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_train = mnist['y_train']
x_test = (mnist['x_test'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_test = mnist['y_test']
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Train Data Set
train_dataset = TensorDataset(torch.FloatTensor(x_train), torch.LongTensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

# Test Data Set
test_dataset = TensorDataset(torch.FloatTensor(x_test), torch.LongTensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [ ]:
class MLP(nn.Module):
 def __init__(self):
  super().__init__()
  self.fc1 = nn.Linear(28*28, 512, bias=True)
  self.fc2 = nn.Linear(512, 512, bias=True)
  self.fc3 = nn.Linear(512, 512, bias=True)
  self.fc4 = nn.Linear(512, 512, bias=True)
  self.fc5 = nn.Linear(512, 10, bias=True)
 def forward(self, x):
  x = F.relu(self.fc1(x)) # layer 1
  x = F.relu(self.fc2(x)) # layer 2
  x = F.relu(self.fc3(x)) # layer 3
  x = F.relu(self.fc4(x)) # layer 4
  x = self.fc5(x) # layer 5
  return x

In [ ]:
from torch.optim import SGD

model = MLP().to(device)
criterion = nn.CrossEntropyLoss().to(device)
opti = SGD(model.parameters(), lr=1e-3)

In [ ]:
epoch = 10
losses = []

for i in range(epoch):
 correct = 0
 for data, target in train_dataloader:
  data = data.view(-1, 28*28).to(device)
 target = target.to(device)

 output = model(data)
 loss = criterion(output, target)
 opti.zero_grad() # gradients을 0으로 초기화
 loss.backward() # backpropagation으로 gradients 계산
 opti.step() # gradients 업데이트
 pred = output.max(1, keepdim=True)[1]
 correct += pred.eq(target.view_as(pred)).sum().item()
 print(100.0 * correct / len(train_dataloader.dataset))


0.0033333333333333335
0.0016666666666666668
0.0033333333333333335
0.005
0.008333333333333333
0.008333333333333333
0.008333333333333333
0.0016666666666666668
0.0033333333333333335
0.0033333333333333335


In [ ]:
correct = 0

for data, target in test_dataloader:
 data = data.view(-1, 28*28).to(device)
 target = target.to(device)
 output = model(data)
 pred = output.max(1, keepdim=True)[1]
 correct += pred.eq(target.view_as(pred)).sum().item()
print(100.0 * correct / len(test_dataloader.dataset))


10.24


In [ ]:
class MLP(nn.Module):
 def __init__(self):
  super().__init__()
  self.fc1 = nn.Linear(28*28, 512, bias=False)
  self.fc2 = nn.Linear(512, 512, bias=False)
  self.fc3 = nn.Linear(512, 10, bias=False)
  self.bn1 = nn.BatchNorm1d(512)
  self.bn2 = nn.BatchNorm1d(512)

  self.dropout1 = nn.Dropout(0.2)
  self.dropout2 = nn.Dropout(0.2)
 def forward(self, x):
  x = F.relu(self.bn1(self.fc1(x))) # layer 1
  x = self.dropout1(x)
  x = F.relu(self.bn2(self.fc2(x))) # layer 2
  x = self.dropout2(x)
  x = self.fc3(x) # layer 3
  return x


In [ ]:
epoch = 20
model.train() # 학습 모드로 변경
for i in range(epoch):
 correct = 0
 for data, target in train_dataloader:
  data = data.view(-1, 28*28).to(device)
  target = target.to(device)

 output = model(data)
 loss = criterion(output, target)
 opti.zero_grad() # gradients을 0으로 초기화
 loss.backward() # backpropagation으로 gradients 계산
 opti.step() # gradients 업데이트
 pred = output.max(1, keepdim=True)[1]
 correct += pred.eq(target.view_as(pred)).sum().item()
 print(100.0 * correct / len(train_dataloader.dataset))


0.005
0.005
0.0033333333333333335
0.01
0.0033333333333333335
0.0016666666666666668
0.008333333333333333
0.0033333333333333335
0.006666666666666667
0.005
0.005
0.015
0.005
0.01
0.006666666666666667
0.008333333333333333
0.01
0.005
0.005
0.006666666666666667


In [ ]:
correct = 0
model.eval() # 평가(테스트) 모드로 변경
for data, target in test_dataloader:
 data = data.view(-1, 28*28).to(device)
 target = target.to(device)
 output = model(data)
 pred = output.max(1, keepdim=True)[1]
 correct += pred.eq(target.view_as(pred)).sum().item()
print(100.0 * correct / len(test_dataloader.dataset))


10.31
